# Exploring conformational space of selected macrocycles - "M13"

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [80]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [81]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Fri Mar 31 15:24:06 2017


In [82]:
# Functions used in this notebook:
def grep_energies_from_sdf_outputs(files):
    energies = []
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies.append(float(lines[i+1]))
    return energies

### Crystal structure of "M13" macrocycle

In [83]:
cm = open('/home/gosia/work/work_on_gitlab/icho/calcs/m13/m13_crystal.xyz','r').read()
vcm = py3Dmol.view(width=400,height=400)
vcm.removeAllModels()
vcm.addModel(cm,'xyz')
vcm.setStyle({'stick':{'radius':0.15,'color':'spectrum'}})
vcm.setBackgroundColor('0xeeeeee')
vcm.zoomTo()
vcm.show()

In [84]:
# decide what is the "core" - a part of molecule, which we wish to be most aligned (rmsd-wise) among all the structures
# here: pyridine ring
m = Chem.AddHs(Chem.MolFromSmiles('N1C(=O)c2cc(C(=O)NCCC(C)CCNC(=O)c3cc(C(=O)NCCCCC1)cc(c3)C(C)(C)C)cc(c2)C(C)(C)C'))
core_m = m.GetSubstructMatch(Chem.MolFromSmiles('c1ccccc1'))

In [85]:
templ_m = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal.sdf')
m_crystal = templ_m[0]

### Conformers generated with the Balloon software:

Conformers were generated in two ways (genetic algorithm):

* Starting with the crystal geometry kept as a template, output: "m13_b_crystal" on the left fig. below

* Starting with the SMILES signature of M13 and allowing to "rebuild the geometry" (option --rebuildGeometry), output: "m13_b_smiles" on the right fig. below

In both cases the Balloon software was asked for 20 conformers (with other parameters set to default values) and it found 15 conformers in the former case and 11 in the latter.

In [86]:
inps_m13_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_crystalsdf/*.sdf')

In [87]:
inps_m13_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_crystalsmiles/*.sdf')

In [88]:
e_m13_b_sdf = grep_energies_from_sdf_outputs(inps_m13_b_sdf)
e_m13_b_smi = grep_energies_from_sdf_outputs(inps_m13_b_smi)
print(len(e_m13_b_smi))

11


In [89]:
%%html
<table>
  <tr>
    <td id="m13_b_crystal" ></td>
    <td id="m13_b_smiles"  ></td>
  <tr>
    <td> m13_b_crystal </td>
    <td> m13_b_smiles  </td>  
  </tr>
</table>

,
m13_b_crystal,m13_b_smiles


In [90]:
# write conformers as a list
allmol_m_b_sdf = []
allmol_m_b_smi = []
suppl_m_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal_sdfout.sdf')
suppl_m_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal_smilesout.sdf')

for mol in suppl_m_b_sdf:
    allmol_m_b_sdf.append(mol)
for mol in suppl_m_b_smi:
    allmol_m_b_smi.append(mol)    

In [91]:
# align:
for mol in allmol_m_b_sdf:
    AllChem.AlignMolConformers(mol,atomIds=core_m)
for mol in allmol_m_b_smi:
    AllChem.AlignMolConformers(mol,atomIds=core_m)    

In [92]:
# view:
p_b_handles=[]

p_b_sdf = py3Dmol.view(width=400,height=400)
for mol in allmol_m_b_sdf:
    mb = Chem.MolToMolBlock(mol)
    p_b_sdf.addModel(mb,'sdf')
p_b_sdf.setStyle({'stick':{'radius':'0.1'}})
p_b_sdf.setBackgroundColor('0xeeeeee')
p_b_sdf.zoomTo()    
p_b_handles.append(p_b_sdf)

p_b_smi = py3Dmol.view(width=400,height=400)
for mol in allmol_m_b_smi:
    mb = Chem.MolToMolBlock(mol)
    p_b_smi.addModel(mb,'sdf')
p_b_smi.setStyle({'stick':{'radius':'0.1'}})
p_b_smi.setBackgroundColor('0xeeeeee')
p_b_smi.zoomTo()    
p_b_handles.append(p_b_smi)

In [93]:
p_b_handles[0].insert('m13_b_crystal')

In [94]:
p_b_handles[1].insert('m13_b_smiles')

Still, the conformers are very much alike. The energy (preoptimized with MM) and the RMSD calculated against the crystal structure is (for the "m13_b_crystal" and the "m13_b_smiles", respectively):

In [95]:
# calculate RMSD against the crystal structure /check that/:
for i, mol in enumerate(allmol_m_b_sdf):
    name = "m13_b_sdf_"+str(i)
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(name, e_m13_b_sdf[i], AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m_crystal))))

name = m13_b_sdf_0, E = 123.621792, RMS = 0.877066
name = m13_b_sdf_1, E = 123.741071, RMS = 2.478622
name = m13_b_sdf_2, E = 112.637348, RMS = 2.516412
name = m13_b_sdf_3, E = 122.191116, RMS = 2.782971
name = m13_b_sdf_4, E = 123.925833, RMS = 2.224798
name = m13_b_sdf_5, E = 125.798216, RMS = 2.471335
name = m13_b_sdf_6, E = 121.413870, RMS = 2.616399
name = m13_b_sdf_7, E = 123.049386, RMS = 1.606641
name = m13_b_sdf_8, E = 123.026848, RMS = 2.128462
name = m13_b_sdf_9, E = 124.181713, RMS = 2.499548
name = m13_b_sdf_10, E = 123.713041, RMS = 2.137413
name = m13_b_sdf_11, E = 121.173757, RMS = 2.372434
name = m13_b_sdf_12, E = 121.467767, RMS = 2.714899
name = m13_b_sdf_13, E = 119.636517, RMS = 2.100488
name = m13_b_sdf_14, E = 121.600070, RMS = 1.907267


In [96]:
# calculate RMSD against the crystal structure /check that/:
for i, mol in enumerate(allmol_m_b_smi):
    name = "m13_b_smi_"+str(i)
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(name, e_m13_b_smi[i], AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m_crystal))))

name = m13_b_smi_0, E = 130.939955, RMS = 1.543718
name = m13_b_smi_1, E = 129.505821, RMS = 2.096618
name = m13_b_smi_2, E = 130.489391, RMS = 2.128287
name = m13_b_smi_3, E = 126.459585, RMS = 1.832656
name = m13_b_smi_4, E = 131.423063, RMS = 2.545563
name = m13_b_smi_5, E = 125.885041, RMS = 2.858193
name = m13_b_smi_6, E = 120.570942, RMS = 2.199209
name = m13_b_smi_7, E = 134.023181, RMS = 2.245091
name = m13_b_smi_8, E = 127.481759, RMS = 2.562983
name = m13_b_smi_9, E = 131.260001, RMS = 2.703377
name = m13_b_smi_10, E = 133.192078, RMS = 1.852128
